In [1]:
import pandas
import numpy as np
import json
import glob
import seaborn
import gab

import re
import collections
import urllib
import datetime
import dateutil

import pandas
import matplotlib.pyplot as plt
import numpy as np
import seaborn

import ipywidgets as widgets
from IPython.display import display

import json

shootingDate = pandas.Timestamp('2018-10-27')

posts = glob.glob('../2019-03-28_processed_3/posts*')
users = glob.glob('../2019-03-28_processed_3/nodes*')

In [2]:
def loadFile(filePath):
    #print(f"Reading: {filePath}")
    targets = []
    with open(filePath) as f:
        for i, l in enumerate(f):
            targets.append(json.loads(l))
            if i % 10000 == 0:
                print(f"{filePath}:{i}", end = '\r')
    return pandas.DataFrame(targets)

In [3]:
def drawCDF(a, ax = None, label = None, ccdf = False, minVal = 0, rescaleX = False, c= None, linewidth = 1):
    if ax is None:
        fig, ax = plt.subplots(figsize = (10, 7))
    a_sort = np.sort(a[a >= minVal])
    p = np.arange(len(a_sort)) / (len(a_sort) - 1)
    if rescaleX:
        a_sort = a_sort / np.max(a_sort)
    if ccdf:
        ax.plot(a_sort, 1 - p, label = label, c = c, linewidth = linewidth)
    else:
        ax.plot(a_sort, p, label = label, c = c, linewidth = linewidth)
    return ax

In [4]:
print("Loading data")
with multiprocessing.Pool(48) as pool:
    print("Loading posts")
    postsDat = pool.map(loadFile, posts)
    print("Loading users")
    usersDat = pool.map(loadFile, users)

Loading data
Loading posts
Loading users


In [ ]:
df_users = pandas.concat(usersDat, sort=False)
df_posts = pandas.concat(postsDat, sort=False)

In [ ]:
with multiprocessing.Pool(12) as pool:
    anti_counts = pool.map(gab.antisemticCount, df_posts['body'])

In [ ]:
df_posts['anti_counts'] = anti_counts
df_posts['anti_counts'] = df_posts['anti_counts'].apply(lambda x : min(x, 1.0))

In [ ]:
df_posts['is_repost'] = df_posts['creator'] == df_posts['orginal_auth']
df_posts['month'] = df_posts['created_at'].apply(lambda x : x[:7])
df_posts['before'] = df_posts['month'].apply(gab.isBefore)

In [ ]:
df_posts_before = df_posts[df_posts['before']].copy()
df_posts_after = df_posts[df_posts['before']==False].copy()

In [ ]:
df_users['create'] = pandas.to_datetime(df_users['created_at_month_label'])

In [ ]:
df_users['before'] = df_users['create'].apply(lambda x : x < shootingDate)

In [ ]:
df_posts_before_grouped = df_posts_before[['creator','body']].groupby('creator').count()
len(df_posts_before_grouped)

266818

In [ ]:
df_posts_after_grouped = df_posts_after[['creator','body']].groupby('creator').count()
len(df_posts_after_grouped)

81713

In [ ]:
df_posts_grouped = df_posts[['creator','body']].groupby('creator').count()
len(df_posts_grouped)

300126

In [ ]:
df_posts_grouped['body_ceil'] = df_posts_grouped['body'].apply(lambda x : x if x < 100 else 100)

In [ ]:
df_posts_grouped['body'].mean()

111.31490773874972

In [ ]:
df_posts_before['body'].mean()

In [ ]:
df_posts_after['body'].mean()

In [ ]:
len(df_posts_before)

In [ ]:
len(df_posts_after)

In [ ]:
len(df_posts)

In [ ]:
post_counts = df_posts.groupby(['month']).count()
create_counts = df_users.groupby(['create']).count()

In [ ]:
post_counts.index = pandas.to_datetime(post_counts.index)

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize = (10, 7), nrows = 2, sharex = True)
create_counts['bio'].plot(ax=ax1)
ax1.axvline(shootingDate,color='r')
post_counts['body'].plot(ax=ax2)
ax2.axvline(shootingDate,color='r')
ax2.set_xlabel("Month")
ax1.set_ylabel("Number of new users")
ax2.set_ylabel("Number of posts")
ax1.set_title("New accounts per month")
ax2.set_title("Posts per month")
plt.savefig("Before_and_after_counts.pdf", format = 'pdf')

In [ ]:
df_posts_before['month'].max()

In [ ]:
df_posts['month'] [:10]